<a href="https://colab.research.google.com/github/ThisIsJorgeLima/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/JAL_LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data


--2019-11-04 22:08:01--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.3’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.1s    

2019-11-04 22:08:02 (127 KB/s) - ‘house-votes-84.data.3’ saved [18171/18171]



In [0]:
import pandas as pd

column_headers = ['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime',
                          'duty-free','south-africa']

df = pd.read_csv('house-votes-84.data', 
                 header=None, 
                 names=column_headers,
                 na_values="?")

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [0]:
# Lets clean up these column '-'
df.columns = [column.replace('-', ' ') for column in df]

In [0]:
df=df.replace({'y':1,'n':0})
df.head()

,party,handicapped infants,water project,budget,physician fee freeze,el salvador aid,religious groups,anti satellite ban,aid to contras,mx missile,immigration,synfuels,education,right to sue,crime,duty free,south africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
df['party'].value_counts()

democrat      267
republican    168
Name: party, dtype: int64

In [0]:
# This isn't loading properly... Lets try dems
# error was fixed. cont. 
rep = df[df['party']=='republican']
rep.head()

,party,handicapped infants,water project,budget,physician fee freeze,el salvador aid,religious groups,anti satellite ban,aid to contras,mx missile,immigration,synfuels,education,right to sue,crime,duty free,south africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [0]:
# Same... Not sure why I'm getting this user/local/error.
# error was fixed.
dem = df[df['party']=='democrat']
dem.head()

,party,handicapped infants,water project,budget,physician fee freeze,el salvador aid,religious groups,anti satellite ban,aid to contras,mx missile,immigration,synfuels,education,right to sue,crime,duty free,south africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [0]:
# the percentage of republicans who voted "yes" (1)
# on the el-salvador-aid

rep['el salvador aid'].sum()/len(rep)

0.9345238095238095

In [0]:
# Lets remove NaN values from our column

col = rep['el salvador aid']

np.isnan(col)


el_salvador_aid_no_nans = col[~np.isnan(col)]

el_salvador_aid_no_nans

el_salvador_aid_no_nans.sum()/len(el_salvador_aid_no_nans)



0.9515151515151515

In [0]:
# Average rate of voting 'yes' or '1' on el salvador aid
rep['el salvador aid'].mean()

0.9515151515151515

In [0]:
rep.isnull().sum()

party                    0
handicapped infants      3
water project           20
budget                   4
physician fee freeze     3
el salvador aid          3
religious groups         2
anti satellite ban       6
aid to contras          11
mx missile               3
immigration              3
synfuels                 9
education               13
right to sue            10
crime                    7
duty free               12
south africa            22
dtype: int64

In [0]:
dem.isnull().sum()

party                    0
handicapped infants      9
water project           28
budget                   7
physician fee freeze     8
el salvador aid         12
religious groups         9
anti satellite ban       8
aid to contras           4
mx missile              19
immigration              4
synfuels                12
education               18
right to sue            15
crime                   10
duty free               16
south africa            82
dtype: int64

# **El Salvador Aid (two-sample t-test)**

1. ***Null Hypothesis***: There is a difference between average voting.
regarding the El Salvador Aid bill between Republicans vs. Democrats
in The House of Representatives. 

2. ***Alternative Hypothesis***:
Level of support between the two parties parties will differ.

3. 95% Confidence Level

In [0]:
# Average rate of voting 'yes' or '1' on el salvador aid
from scipy.stats import ttest_ind
rep['el salvador aid'].mean()

0.9515151515151515

In [0]:
# Average rate of voting 'yes' or '1' on el salvador aid
dem['el salvador aid'].mean()

0.21568627450980393

In [0]:
ttest_ind(rep['el salvador aid'], dem['el salvador aid'], nan_policy='omit')

Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)

In [0]:
# Remove NaN values from the column Dems column
# since we removed form Rep already:


col = rep['el salvador aid']
rep_el_salvador_aid_no_nans = col[~np.isnan(col)]

col = dem['el salvador aid']
dem_el_salvador_aid_no_nans = col[~np.isnan(col)]


# sample sizes for the samples:
print(len(rep_el_salvador_aid_no_nans))
print(len(dem_el_salvador_aid_no_nans))

165
255


**In Conclusion**

Lets determine degrees of freedom

So in this case, df = 166-1 = 165

4. T-statistic: 21.13
5. P-value: 5.6


1. ***Null Hypothesis*** average rep don't support 

In [0]:
df.head()

,party,handicapped infants,water project,budget,physician fee freeze,el salvador aid,religious groups,anti satellite ban,aid to contras,mx missile,immigration,synfuels,education,right to sue,crime,duty free,south africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


1a.***Null Hypothesis:***

$\bar{x}_{1}$ (average dem support for Education bill) == 1
 This states that 100% of democrats support this bill. Full support.

1b. ***Null Hypothesis:***

$\bar{x}_{1}$ (average dem support for Education bill) == .5
this states that 50% of democrats support this bill. The party is split.

1c. ***Null Hypothesis:***

$\bar{x}_{1}$ (average dem support for Education bill) == 0
This states that 0% of democrats support the following bill.

1d. ***Null Hypothesis:***

$\bar{x}_{1}$ (average dem support SA bill) == .78245
This states that 0% of democrats support this bill.
In addition, the party is against the bill.

**With 1-sample t-tests we will frame what were asking through our
choices of our null hypothesis:**

1. Null Hypothesis:  𝑥¯1  (average dem support for Education bill) == .5

   This states that 50% of democrats support this bill and the party is split.

2. Alternative Hypothesis: Support is not equal to .5 or 50%

   𝑥¯1  (average dem support for Education bill)  ≠  .5

  This states nothing regarding if the support is greater than or less than 50%,  it's replying that it's not 50% - that its different and its something other    than 50%.

3. Confidence Level: 95%

In [0]:
from scipy.stats import ttest_1samp

In [0]:
dem['education'].sum()/len(dem)

0.1348314606741573

In [0]:
dem_budget_no_nan = col[~np.isnan(col)]

print(len(dem_budget_no_nan))

157


In [0]:
dem['education'].mean()

0.14457831325301204

In [0]:
ttest_1samp(dem['education'],.5, nan_policy='omit')

Ttest_1sampResult(statistic=-15.915784824279635, pvalue=8.900198902856228e-40)

In [0]:
dem['education'].sum()/len(rep)

0.21428571428571427

In [0]:
ttest_1samp(dem['education'], .9, nan_policy='omit')


Ttest_1sampResult(statistic=-33.82778672821468, pvalue=6.837447861906306e-95)

In [0]:
ttest_1samp(dem['education'], .899, nan_policy='omit')

Ttest_1sampResult(statistic=-33.78300672345485, pvalue=8.958307039436988e-95)

Since the p-value of the t-test is less than the threshold
we are going to reject the null hypothesis.